<a href="https://colab.research.google.com/github/jbwenjoy/000545000/blob/main/CIS_545_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CIS 5450 Final Project - Spring 2025**
## Group Member:
Yiyan Liang: edgarl@seas.upenn.edu
<br>Bowen Jiang: jbwenjoy@seas.upenn.edu
<br>Binglong Bao: binglong@seas.upenn.edu

In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_path_in_drive = '/content/drive/MyDrive/CIS545-final/'
flights_data = '/content/drive/MyDrive/CIS545-final/US Airline Flight Routes and Fares 1993-2024.csv'
fuel_data = '/content/drive/MyDrive/CIS545-final/flat-ui__data-Fri Mar 28 2025.csv'
kaggle_key = '/content/drive/MyDrive/CIS545-final/kaggle.json'

# Check if these files exists
file_paths = [
    (data_path_in_drive, "Directory"),
    (flights_data, "File"),
    (fuel_data, "File"),
    (kaggle_key, "File")
]

for path, file_type in file_paths:
    if not os.path.exists(path):
        if file_type == "Directory":
            os.makedirs(path)
            print(f"Directory '{path}' created successfully.")
        else:
            print(f"{file_type} '{path}' does not exist.")
    else:
        print(f"{file_type} '{path}' exists.")


Directory '/content/drive/MyDrive/CIS545-final/' exists.
File '/content/drive/MyDrive/CIS545-final/US Airline Flight Routes and Fares 1993-2024.csv' exists.
File '/content/drive/MyDrive/CIS545-final/flat-ui__data-Fri Mar 28 2025.csv' exists.
File '/content/drive/MyDrive/CIS545-final/kaggle.json' exists.


# **1. Introduction and Background**

# **2. Get to know data**

# **3. EDA**

In [15]:
import pandas as pd
from scipy import stats

In [5]:
# Importing CSV file
df_original = pd.read_csv(flights_data, low_memory=False)

In [6]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245955 entries, 0 to 245954
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tbl             245955 non-null  object 
 1   Year            245955 non-null  int64  
 2   quarter         245955 non-null  int64  
 3   citymarketid_1  245955 non-null  int64  
 4   citymarketid_2  245955 non-null  int64  
 5   city1           245955 non-null  object 
 6   city2           245955 non-null  object 
 7   airportid_1     245955 non-null  int64  
 8   airportid_2     245955 non-null  int64  
 9   airport_1       245955 non-null  object 
 10  airport_2       245955 non-null  object 
 11  nsmiles         245955 non-null  int64  
 12  passengers      245955 non-null  int64  
 13  fare            245955 non-null  float64
 14  carrier_lg      244415 non-null  object 
 15  large_ms        244415 non-null  float64
 16  fare_lg         244415 non-null  float64
 17  carrier_lo

In [7]:
df_original.duplicated().sum()

np.int64(0)

In [8]:
df_original.head()

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk
0,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,...,81.43,G4,1.0000,81.43,G4,1.0000,81.43,NaN,NaN,202131013514112ABEPIE
1,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,...,208.93,DL,0.4659,219.98,UA,0.1193,154.11,NaN,NaN,202131013515304ABETPA
2,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,...,184.56,WN,0.9968,184.44,WN,0.9968,184.44,NaN,NaN,202131014011259ABQDAL
3,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,...,182.64,AA,0.9774,183.09,AA,0.9774,183.09,NaN,NaN,202131014011298ABQDFW
4,Table1a,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,...,177.11,WN,0.6061,184.49,AA,0.3939,165.77,NaN,NaN,202131014014107ABQPHX


In [9]:
df_original.sample(3)

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk
241202,Table1a,2023,3,31453,31703,"Houston, TX","New York City, NY (Metropolitan Area)",12191,12478,HOU,...,243.18,DL,0.4868,288.84,AA,0.4342,192.05,NaN,NaN,202331219112478HOUJFK
81619,Table 1a,2008,1,30189,30466,"Colorado Springs, CO","Phoenix, AZ",11109,14107,COS,...,162.71,US,0.8900,159.18,DL,0.0500,151.07,"Colorado Springs, CO\n(38.835224, -104.819798)","Phoenix, AZ\n(30.406931, -87.217578)",200811110914107COSPHX
223870,Table1a,2020,3,31423,32575,"Des Moines, IA","Los Angeles, CA (Metropolitan Area)",11423,13891,DSM,...,188.08,AA,0.5529,190.31,UA,0.1346,170.07,NaN,NaN,202031142313891DSMONT


In [10]:
print(df_original.shape)

df_clean = df_original.copy()
df_clean = df_clean.dropna()
df_clean = df_clean.dropna(axis=1)
df_clean = df_clean.drop_duplicates()

print(df_clean.shape)

(245955, 23)
(205189, 23)


In [17]:
# Calculate Z-scores of each value in the DataFrame
z_scores = stats.zscore(df_clean.select_dtypes(include=['float64', 'int64']))

# Identify outliers (Z-score > 3 or < -3)
df_outliers = df_clean[(z_scores > 3).any(axis=1) | (z_scores < -3).any(axis=1)]
print(df_outliers.shape)

# Remove rows with outliers based on Z-score
df_clean = df_clean[(z_scores < 3).all(axis=1) & (z_scores > -3).all(axis=1)]
df_clean.shape


(9513, 23)


(195676, 23)

# **4. Modeling**

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, FunctionTransformer
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import TransformerMixin, BaseEstimator

In [18]:
df = df_clean.copy()
df['quaters'] = df['quarter'].astype(int) + (df['Year'].astype(int) - 2000) * 4
new_df = df[['fare', 'nsmiles', 'quaters']].dropna()

X = new_df[['nsmiles', 'quaters']]
y = new_df['fare']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [20]:
log_transformer = FunctionTransformer(np.log1p, inverse_func=np.expm1, validate=True)

pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', LinearRegression()),
])

pipe.fit(X_train, y_train)
print(f"R^2 socre: {pipe.score(X_test, y_test)}")
print(f"MSE socre: {mean_squared_error(y_test, pipe.predict(X_test))}")
print(f"model intercept: {pipe.named_steps['model'].intercept_}")
print(f"model coefficients: {pipe.named_steps['model'].coef_}")

def plot_prediction(model,X_test, y_test):
    y_test_pred = model.predict(X_test)
    # use plotly to plot y_train_pred and y_test_pred vs y_train and y_test
    fig = go.Figure()
    # control figure size
    fig.update_layout(width=800, height=800)
    fig.add_trace(go.Scatter(x=y_test, y=y_test_pred, mode='markers', name='test'))
    min_val = min(y_test.min(), y_test_pred.min())
    max_val = max(y_test.max(), y_test_pred.max())
    fig.update_xaxes(range=[min_val - 50, max_val + 50])
    fig.update_yaxes(range=[min_val - 50, max_val + 50])
    # add title and axis labels
    fig.update_layout(title='Prediction vs Actual',
                    xaxis_title='Actual',
                    yaxis_title='Prediction')
    fig.add_trace(go.Scatter(
                    x=[min_val, max_val],
                    y=[min_val, max_val],
                    mode='lines',
                    name='Ideal (y = x)',
                    line=dict(color='red', dash='dash')
    ))
    fig.show()
    return



# sample some points from X_test and y_test
X_test_sample = X_test.sample(100)
y_test_sample = y_test.loc[X_test_sample.index]
# plot the prediction
plot_prediction(pipe, X_test_sample, y_test_sample)



R^2 socre: 0.30598937368672985
MSE socre: 3328.0376524465564
model intercept: 132.99723359076805
model coefficients: [135.42843055  47.25294866]


As we can see, the baseline linear regression model has a very low R-squared value. This means that the model is not a good fit for the data.
We would like to further improve the performance by :
1. Adding more features, such as dealing with the categorical features and using the one-hot encoding technique.
2. Using more advanced machine learning algorithms, such as **kernel regression** and **gradient boosting trees**. These algorithms are designed to handle complex relationships between the features and the target variable. For example, kernel regression uses a kernel function to map the input features to a higher-dimensional space, while gradient boosting trees use a series of decision trees to build a strong predictive model. These algorithms can capture more complex relationships between the features and the target variable, which can lead to better performance than linear regression does.
3. Tuning the hyperparameters of the machine learning algorithms to improve their performance.

# **5. Project Management**

# **6. Hypothesis Testing**